# Environmental Conditions in Vienna

[![Binder](http://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/anitagraser/ogd-at-lab/main?urlpath=lab/tree/notebooks/environment.ipynb)




In [ ]:
import hvplot.pandas
import pandas as pd
import geopandas as gpd
from utils.dataaccess import gdf_from_wfs, get_weather_df, get_heatvulnerabilityindex_gdf

## Urban Heat

Dataset: [Urban Heat Vulnerability Index (UHVI)](https://www.data.gv.at/katalog/dataset/urban-heat-vulnerability-index-uhvi-wien)

In [ ]:
heatvulidx = get_heatvulnerabilityindex_gdf()

In [ ]:
heatvulidx.hvplot(geo=True, tiles='OSM', c='AVG_UHVI_A', title='Average Urban Heat Vulnerability Index - All age groups').opts(active_tools=['wheel_zoom'])

## Air Quality

In [ ]:
stations = gdf_from_wfs('LUFTGUETENETZOGD')
stations.head()

In [ ]:
weather = get_weather_df()
weather